In [ ]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

In [ ]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("PGA_data_full").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://red-team-final.s3.amazonaws.com/reshaped_full_cleaned_dataset.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("reshaped_full_cleaned_dataset.csv"), sep=",", header=True, inferSchema=True)
df.show()

In [ ]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://red-team-final.c3yshvuxvm9x.us-east-2.rds.amazonaws.com:5432/red-team"
config = {"user":"postgres", 
          "password": "databootcamp1", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write to table in RDS
df.write.jdbc(url=jdbc_url, table='pga_stats_full', mode=mode, properties=config)